In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm_notebook
from time import sleep
import pandas as pd
import re

In [2]:
driver_path = "driver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)

# url 수집

In [3]:
months = ['03', '04', '05', '06', '07', '08', '09', '10']
url_list = []
date_list = []

for month in months:
    url_schedule = 'https://sports.news.naver.com/kbaseball/schedule/index.nhn?month={month}&year=2018'
    driver.get(url_schedule.format(month=month))
    html_sch = driver.page_source
    soup_sch = BeautifulSoup(html_sch, "lxml")
    game_url = soup_sch.find("div", "tb_wrap").find_all("a")
    for j in range(len(game_url)):
        url_list.append(game_url[j].attrs['href'])
        
url_list = pd.DataFrame({'url':url_list})
url_list = url_list[url_list.url.str.contains('gameResult') == True]
url_list = url_list[url_list.url > '/gameCenter/gameResult.nhn?category=kbo&gameId=20180321LGWO02018']
url_list = url_list[url_list.url < '/gameCenter/gameResult.nhn?category=kbo&gameId=33331019WOHH02018']
url_list = list(url_list.url)

for i in range(len(url_list)):
    date = re.search('\d\d\d\d\d\d\d\d', url_list[i]).group()
    date_list.append(date)

# 기록 수집

In [20]:
table = []
loc = []
teams_H = []
teams_P = []
date_list = []

for url in url_list:
    url_page = 'https://sports.news.naver.com{url}'
    driver.get(url_page.format(url=url))
    html = driver.page_source # 드라이버에서 현재 띄우고 있는 html을 주소 변수로 설정
    soup = BeautifulSoup(html, "lxml")
    view = soup.find("div", "ground_area")
    hit = view.find_all('div','ico hit')
    out = view.find_all('div','ico out')
    hrun = view.find_all('div','ico run')
    ha = re.findall('[A-Z][A-Z]', url)

    for i in range(len(hit)):
        table.append(hit[i].find('div').get_text())
    for i in range(len(out)):
        table.append(out[i].find('div').get_text())
    for i in range(len(hrun)):
        table.append(hrun[i].find('div').get_text())
    
    for i in range(len(hit)):
        loc.append(hit[i].attrs['style'])
    for i in range(len(out)):
        loc.append(out[i].attrs['style'])
    for i in range(len(hrun)):
        loc.append(hrun[i].attrs['style'])
    
    for i in range(len(hit)):
        if hit[i].attrs['id'].split('_')[0] == 'home':
            team = ha[1]
        else:
            team = ha[0]
        teams_H.append(team)
    for i in range(len(out)):
        if out[i].attrs['id'].split('_')[0] == 'home':
            team = ha[1]
        else:
            team = ha[0]
        teams_H.append(team)
    for i in range(len(hrun)):
        if hrun[i].attrs['id'].split('_')[0] == 'home':
            team = ha[1]
        else:
            team = ha[0]
        teams_H.append(team)
    
    for i in range(len(hit)):
        if hit[i].attrs['id'].split('_')[0] == 'home':
            team = ha[0]
        else:
            team = ha[1]
        teams_P.append(team)
    for i in range(len(out)):
        if out[i].attrs['id'].split('_')[0] == 'home':
            team = ha[0]
        else:
            team = ha[1]
        teams_P.append(team)
    for i in range(len(hrun)):
        if hrun[i].attrs['id'].split('_')[0] == 'home':
            team = ha[0]
        else:
            team = ha[1]
        teams_P.append(team)
        
    for i in range(len(hit)+len(out)+len(hrun)):
        date_list.append(re.search('\d\d\d\d\d\d\d\d', url).group())
    
    sleep(5/10);
    

In [21]:
df = pd.DataFrame({'date':date_list, 'batter_team':teams_H, 'pitcher_team':teams_P, 'table':table, 'loc':loc})
df['X'] = df['loc'].apply(lambda e: e.split(';')[0])
df['Y'] = df['loc'].apply(lambda e: e.split(';')[1])
df = df.drop(columns = 'loc')

df['fr'] = df['table'].apply(lambda e: e.split('(')[0])
df['inning'] = df['fr'].apply(lambda e: e.split(' ')[0])
df['batter'] = df['fr'].apply(lambda e: e.split(' ')[1])
df['result'] = df['fr'].apply(lambda e: e.split(' ')[2])
df['ba'] = df['table'].apply(lambda e: e.split('(')[1])
df['cause'] = df['ba'].apply(lambda e: e.split(')')[0])
df['pitcher'] = df['ba'].apply(lambda e: e.split(')')[1])
df = df.drop(columns = ['table', 'fr', 'ba'])

df['X'] = df['X'].str.replace("left:", "")
df['X'] = df['X'].str.replace("px", "")
df['Y'] = df['Y'].str.replace("top:", "")
df['Y'] = df['Y'].str.replace("px", "")
df['pitcher'] = df['pitcher'].str.replace("상대투수-", "")

record = df[['date', 'inning', 'batter_team', 'batter', 'pitcher_team', 'pitcher', 'result', 'cause', 'X', 'Y']]
record = record.replace({'WO':'넥센', 'HH':'한화', 'HT':'KIA', 'OB':'두산', 'LT':'롯데', 'SS':'삼성'})

In [24]:
record.head()

,date,inning,batter_team,batter,pitcher_team,pitcher,result,cause,X,Y
0,20180324,2회말,넥센,김하성,한화,샘슨,안타,좌중간 1루타,199.36,159.5
1,20180324,3회말,넥센,박병호,한화,샘슨,안타,좌익수 앞 1루타,150.37,181.25
2,20180324,4회말,넥센,고종욱,한화,샘슨,안타,우익수 앞 1루타,300.18,174.5
3,20180324,4회말,넥센,이정후,한화,샘슨,안타,좌중간 2루타,148.95,100.25
4,20180324,4회말,넥센,박동원,한화,샘슨,안타,중견수 왼쪽 1루타,207.17,131
